In [1]:
from dotenv import load_dotenv
load_dotenv()

import requests
from io import BytesIO
import geopandas as gpd
import pandas as pd
import json
import rasterio
from rasterio.features import shapes


import os
from tqdm import tqdm

from src.shared.libs.s3 import S3Bucket

BASE_URL="https://ies-ows.jrc.ec.europa.eu/effis"

DEFAULT_ARGS = {
    "LAYERS": "mf010.fwi",
    "FORMAT": "image/tiff",
    "TRANSPARENT": "true",
    "SINGLETILE": "false",
    "SERVICE": "wms",
    "VERSION": "1.1.1",
    "REQUEST": "GetMap",
    "STYLES": "",
    "SRS": "EPSG:4326",
    "BBOX": "-5.0,43.0,10.0,52.0",
    "WIDTH": "1600",
    "HEIGHT": "1200",
    "TIME": "2021-07-01"
}

S3 = S3Bucket(
    bucket_name="fwi",
    endpoint_url=os.getenv("S3_ENDPOINT_URL"),
    region_name=os.getenv("AWS_REGION_NAME"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

def get_fwi_image(date):
    args = DEFAULT_ARGS.copy()
    args["TIME"] = date
    url = BASE_URL + "?" + "&".join([f"{k}={v}" for k, v in args.items()])
    response = requests.get(url, stream=True)

    return response

def process(response):
    with rasterio.open(BytesIO(response.content)) as src:
        image = src.read(1)  # first band
        results = (
            {"properties": {"fwi_pixel_value": v}, "geometry": s}
            for i, (s, v) in enumerate(
                shapes(image, transform=src.meta["transform"])
            )
        )

        gpd_polygonized_raster = gpd.GeoDataFrame.from_features(
            list(results), crs=src.meta["crs"]
        ).query("fwi_pixel_value != 0")

    return gpd_polygonized_raster

def gpd_to_json(geodataframe: gpd.GeoDataFrame):
    json_fwi = geodataframe.to_json()
    json_fwi = json.loads(json_fwi)
    for feature_dict in json_fwi["features"]:
        del feature_dict["id"]
    return {
            "type": "FeatureCollection",
            "crs": {
                "type": "name",
                "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"},
            },
            "features": json_fwi["features"],
        }

def main(date):
    return gpd_to_json(process(get_fwi_image(date)))

In [12]:
list_dates = pd.date_range(start="2024-01-02", end="2024-04-11", freq="D").strftime("%Y-%m-%d")

for date in tqdm(list_dates):
    year, month, day = date.split("-")
    S3.write_json_to_s3(
        object_key=f"year={year}/month={month}/day={day}/fwi_values.json",
        json_data=main(date),
    )

100%|██████████| 101/101 [03:12<00:00,  1.91s/it]
